In [ ]:
!pip install geopandas

     |████████████████████████████████| 994 kB 7.9 MB/s 
     |████████████████████████████████| 6.6 MB 40.6 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Data Analyst/Điểm thi đại học 2021/data

/content/drive/MyDrive/Data Analyst/Điểm thi đại học 2021/data


# Preprocess data

In [ ]:
df_2021 = pd.read_csv('./total',  delimiter = "\t")
df_2021.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SBD,Tên,Ngày Sinh,Giới tính,Toán,Văn,Lý,Hoá,Sinh,KHTN,Lịch Sử,Địa Lý,GDCD,KHXH,Ngoại Ngữ
0,1000043,NaN,NaN,NaN,8,8.5,,,,,5.75,6.5,8.75,,8.8
1,1000163,NaN,NaN,NaN,3.8,7.5,,,,,6.75,7,,,
2,1000040,NaN,NaN,NaN,3.8,2,,,,,2.5,4.5,,,
3,1000007,NaN,NaN,NaN,9,5.25,7.25,4.75,3.5,,,,,,9
4,1000180,NaN,NaN,NaN,8.8,8.5,,,,,5.25,7.25,8,,9.6


In [ ]:
# Drop 2 columns KHTN and KHXH
df_2021.drop(columns=['KHTN', 'KHXH'], inplace=True)

# Replace white-space null value with np.nan
for col in df_2021.columns:
    df_2021[col] = df_2021[col].apply(lambda x: np.nan if x == ' ' else x)

In [ ]:
# Extract information from messy value
def extract_info(x):
    res = []
    if x[-1] != ';':
        x = x + ';'
    if x[0] == ';':
        x = x[1:]
    res.append(int(x.split('|')[0]))
    res.extend([np.nan, np.nan, np.nan])
    score_list = x.split('|')[1][:-1]
    for score in score_list.split(';'):
        if score[-1] == ':':
            res.append(np.nan)
        else:
            res.append(score.split(':')[1])
    return res

In [ ]:
# Add missing data from province 13, 14, 29 and 32
extracted_list = []
for file in ['13.txt', '14.txt', '29.txt', '32.txt']:
    for line in list(pd.read_csv('./diemthi2021/' + file, header=None, delimiter = "\t")[0]):
        extracted_list.append(extract_info(line))
    df_2021 = pd.concat([df_2021, pd.DataFrame.from_records(extracted_list, columns=df_2021.columns)])
    extracted_list = []
df_2021.head()

,SBD,Tên,Ngày Sinh,Giới tính,Toán,Văn,Lý,Hoá,Sinh,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ
0,1000043,NaN,NaN,NaN,8,8.5,NaN,NaN,NaN,5.75,6.5,8.75,8.8
1,1000163,NaN,NaN,NaN,3.8,7.5,NaN,NaN,NaN,6.75,7,NaN,NaN
2,1000040,NaN,NaN,NaN,3.8,2,NaN,NaN,NaN,2.5,4.5,NaN,NaN
3,1000007,NaN,NaN,NaN,9,5.25,7.25,4.75,3.5,NaN,NaN,NaN,9
4,1000180,NaN,NaN,NaN,8.8,8.5,NaN,NaN,NaN,5.25,7.25,8,9.6


In [ ]:
# Turn score from object type to float type
for col in ['Toán', 'Văn', 'Lý', 'Hoá', 'Sinh', 'Lịch Sử', 'Địa Lý', 'GDCD', 'Ngoại Ngữ']:
    df_2021[col] = df_2021[col].astype(float)

# Add year column
df_2021['Year'] = 2021

In [ ]:
# Read file diemthi2020
old_df_2020 = pd.read_csv('./diemthi2020.csv')
old_df_2020.head()

,Unnamed: 0,Dia,GDCD,Hoa,Li,Ma_mon_ngoai_ngu,Ngoai_ngu,Sinh,Su,Toan,Van,sbd
0,0,7.00,6.50,NaN,NaN,N1,4.2,NaN,4.75,6.4,6.75,18014547
1,1,7.75,7.75,NaN,NaN,N1,2.8,NaN,3.75,7.6,6.00,18014530
2,2,6.50,NaN,NaN,NaN,NaN,NaN,NaN,4.00,4.8,4.75,18014521
3,3,8.00,9.50,NaN,NaN,N1,5.8,NaN,8.25,8.0,7.00,18014517
4,4,NaN,NaN,8.5,8.0,N1,4.0,5.0,NaN,8.2,6.50,18014523


In [ ]:
# Create dataframe for year 2020
df_2020 = pd.DataFrame()
df_2020['SBD'] = old_df_2020['sbd']
df_2020['Tên'] = np.nan
df_2020['Ngày Sinh'] = np.nan
df_2020['Giới tính'] = np.nan
df_2020['Toán'] = old_df_2020['Toan']
df_2020['Văn'] = old_df_2020['Van']
df_2020['Lý'] = old_df_2020['Li']
df_2020['Hoá'] = old_df_2020['Hoa']
df_2020['Sinh'] = old_df_2020['Sinh']
df_2020['Lịch Sử'] = old_df_2020['Su']
df_2020['Địa Lý'] = old_df_2020['Dia']
df_2020['GDCD'] = old_df_2020['GDCD']
df_2020['Ngoại Ngữ'] = old_df_2020['Ngoai_ngu']
df_2020['Year'] = 2020
df_2020.head()

,SBD,Tên,Ngày Sinh,Giới tính,Toán,Văn,Lý,Hoá,Sinh,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ,Year
0,18014547,NaN,NaN,NaN,6.4,6.75,NaN,NaN,NaN,4.75,7.00,6.50,4.2,2020
1,18014530,NaN,NaN,NaN,7.6,6.00,NaN,NaN,NaN,3.75,7.75,7.75,2.8,2020
2,18014521,NaN,NaN,NaN,4.8,4.75,NaN,NaN,NaN,4.00,6.50,NaN,NaN,2020
3,18014517,NaN,NaN,NaN,8.0,7.00,NaN,NaN,NaN,8.25,8.00,9.50,5.8,2020
4,18014523,NaN,NaN,NaN,8.2,6.50,8.0,8.5,5.0,NaN,NaN,NaN,4.0,2020


In [ ]:
# Create a final dataframe
final_data = pd.concat([df_2020, df_2021]).reset_index(drop=True)
final_data

,SBD,Tên,Ngày Sinh,Giới tính,Toán,Văn,Lý,Hoá,Sinh,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ,Year
0,18014547,NaN,NaN,NaN,6.4,6.75,NaN,NaN,NaN,4.75,7.00,6.50,4.2,2020
1,18014530,NaN,NaN,NaN,7.6,6.00,NaN,NaN,NaN,3.75,7.75,7.75,2.8,2020
2,18014521,NaN,NaN,NaN,4.8,4.75,NaN,NaN,NaN,4.00,6.50,NaN,NaN,2020
3,18014517,NaN,NaN,NaN,8.0,7.00,NaN,NaN,NaN,8.25,8.00,9.50,5.8,2020
4,18014523,NaN,NaN,NaN,8.2,6.50,8.0,8.5,5.0,NaN,NaN,NaN,4.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857872,32008526,NaN,NaN,NaN,5.8,6.00,NaN,NaN,NaN,3.75,5.50,7.50,3.2,2021
1857873,32008527,NaN,NaN,NaN,5.6,6.00,NaN,NaN,NaN,3.00,5.00,6.50,4.8,2021
1857874,32008528,NaN,NaN,NaN,6.6,8.00,NaN,NaN,NaN,4.75,6.50,8.00,4.0,2021
1857875,32008529,NaN,NaN,NaN,6.0,7.25,NaN,NaN,NaN,4.50,7.00,8.50,4.6,2021


In [ ]:
# Clean sbd column
def convert(x):
    x = str(x)
    if len(x) == 7:
        return '0' + x
    return x

final_data['SBD'] = final_data['SBD'].apply(lambda x: convert(x))

In [ ]:
import geopandas as gpd
map_df = gpd.read_file('./diaphantinh.geojson')
map_df.head()

,gid,code,ten_tinh,geometry
0,1,AD01,An Giang,"MULTIPOLYGON (((105.11524 10.95566, 105.11463 ..."
1,2,AD01,Bà Rịa -Vũng Tàu,"MULTIPOLYGON (((106.08110 8.57754, 106.08069 8..."
2,3,AD01,Bắc Giang,"MULTIPOLYGON (((106.16542 21.62022, 106.16925 ..."
3,4,AD01,Bắc Kạn,"MULTIPOLYGON (((105.74420 22.73519, 105.74624 ..."
4,5,AD01,Bạc Liêu,"MULTIPOLYGON (((105.32591 9.60004, 105.32755 9..."


In [ ]:
# Fix some typo errors
map_df.loc[44, 'ten_tinh'] = 'Quảng Bình'
map_df.loc[31, 'ten_tinh'] = 'Kiên Giang'
map_df.loc[12, 'ten_tinh'] = 'Cần Thơ'

In [ ]:
# Create a dictionary to map
code_provinces = ['51', '52', '18', '11', '60', '19', '56', '37', '44', '43', '47', '61', '55',
                  '06', '04', '40', '63', '62', '48', '50', '38', '05', '24', '01', '30', '21',
                  '03', '64', '23', '22', '41', '54', '36', '07', '42', '10', '08', '49', '25',
                  '29', '27', '45', '15', '39', '31', '34', '35', '17', '32', '59', '14', '46',
                  '26', '12', '28', '33', '53', '02', '58', '09', '57', '16', '13']
                  
mapping = {code_provinces[i]: map_df['ten_tinh'].unique().tolist()[i] for i in range(len(code_provinces))}

In [ ]:
# Create columns of code and province
final_data['code'] = final_data['SBD'].str[:2]
final_data['province'] = final_data['code'].apply(lambda x: mapping[x])

In [ ]:
final_data

,SBD,Tên,Ngày Sinh,Giới tính,Toán,Văn,Lý,Hoá,Sinh,Lịch Sử,Địa Lý,GDCD,Ngoại Ngữ,Year,code,province
0,18014547,NaN,NaN,NaN,6.4,6.75,NaN,NaN,NaN,4.75,7.00,6.50,4.2,2020,18,Bắc Giang
1,18014530,NaN,NaN,NaN,7.6,6.00,NaN,NaN,NaN,3.75,7.75,7.75,2.8,2020,18,Bắc Giang
2,18014521,NaN,NaN,NaN,4.8,4.75,NaN,NaN,NaN,4.00,6.50,NaN,NaN,2020,18,Bắc Giang
3,18014517,NaN,NaN,NaN,8.0,7.00,NaN,NaN,NaN,8.25,8.00,9.50,5.8,2020,18,Bắc Giang
4,18014523,NaN,NaN,NaN,8.2,6.50,8.0,8.5,5.0,NaN,NaN,NaN,4.0,2020,18,Bắc Giang
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1857872,32008526,NaN,NaN,NaN,5.8,6.00,NaN,NaN,NaN,3.75,5.50,7.50,3.2,2021,32,Quảng Trị
1857873,32008527,NaN,NaN,NaN,5.6,6.00,NaN,NaN,NaN,3.00,5.00,6.50,4.8,2021,32,Quảng Trị
1857874,32008528,NaN,NaN,NaN,6.6,8.00,NaN,NaN,NaN,4.75,6.50,8.00,4.0,2021,32,Quảng Trị
1857875,32008529,NaN,NaN,NaN,6.0,7.25,NaN,NaN,NaN,4.50,7.00,8.50,4.6,2021,32,Quảng Trị


In [ ]:
final_data.to_csv('./final_data.csv', index=False)